# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA 
* Model: GPT-2
* Evaluation approach: train.eval
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
from transformers import AutoModelForCausalLM

model_name="gpt2"

model = AutoModelForCausalLM.from_pretrained(model_name)

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
from datasets import load_dataset

# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
dataset = {split: dataset for split, dataset in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    dataset[split] = dataset[split].shuffle(seed=57).select(range(500))

# Show the dataset
dataset

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True,padding="max_length"), batched=True)

# Inspect the available columns in the dataset
tokenized_dataset['train'].set_format('torch', columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset['test'].set_format('torch', columns=['label', 'input_ids', 'attention_mask'])
tokenized_dataset["train"]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [3]:
dataset['train'][0]

{'text': 'In the future of 1985, a governmental committee headed by Howard Hesseman, is holding hearings on TV\'s first uncensored network. They sample it\'s programming, that play as a series of skits. I can name the good \'skit\' movies on one hand, not using my thumb. "Amazon Women on the Moon", "Kentucky Fried Movie", "The Meaning of Life", and "Mr. Mike\'s Mondo Video". Notice how I didn\'t mention "Tunnel Vision"? The reason for that is that this \'movie\' is death in cinematic form. None of the skits are even remotely funny, or even the least bit clever. It takes some sort of great ineptitude on the film makers\' part to not even get one laugh out of me.<br /><br />My Grade: F <br /><br />Eye Candy: Dody Dorn goes full frontal',
 'label': 0}

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["text"], truncation=True,padding=True), batched=True)

# Inspect the available columns in the dataset
tokenized_dataset["train"]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    id2label={0: "NEG", 1: "POS"},
    label2id={"NEG": 0, "POS": 1},
    pad_token_id=tokenizer.eos_token_id,  # Set pad token id
)

for param in model.parameters():
    param.requires_grad = True

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import numpy as np
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)
    return {"accuracy": (predictions == labels).mean()}

In [7]:
training_args = TrainingArguments(
    output_dir="./data/movie_sentiment",
    learning_rate=2e-3,
    per_device_eval_batch_size=4,
    per_device_train_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=2,
    weight_decay=0.01,
    load_best_model_at_end=True,
)

In [9]:
trainer_base_model = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

#trainer_base_model.train() This line is wrong because the comparison should be made between original foundation model and PEFT model (NOT between fully fine tuned foundation model and PEFT model.

In [10]:
base_eval=trainer_base_model.evaluate()

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [11]:
# The Lora config was taken from: https://knowledge.udacity.com/questions/1025308

from peft import LoraConfig,TaskType

config = LoraConfig(
    r=8, # Rank
    lora_alpha=32,
    target_modules=['c_attn', 'c_proj'],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

In [12]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

lora_model.print_trainable_parameters()

/opt/conda/lib/python3.10/site-packages/peft/tuners/lora.py:475: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 814,080 || all params: 125,253,888 || trainable%: 0.6499438963523432


In [13]:
# Also used mentor assistant to fix some errors: https://knowledge.udacity.com/questions/1026310

import torch

trainset=tokenized_dataset["train"].rename_column("label", "labels")
testset=tokenized_dataset["test"].rename_column("label", "labels")

trainset.set_format(type="torch", columns=["input_ids", "labels"])
trainset = trainset.remove_columns(["text"])

testset.set_format(type="torch", columns=["input_ids", "labels"])
testset = testset.remove_columns(["text"])

trainset

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 500
})

In [14]:
trainer_lora = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset=testset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
    
)

trainer_lora.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.941003,0.514000
2,No log,0.694959,0.496000


TrainOutput(global_step=250, training_loss=0.8230758666992187, metrics={'train_runtime': 323.6346, 'train_samples_per_second': 3.09, 'train_steps_per_second': 0.772, 'total_flos': 527585771520000.0, 'train_loss': 0.8230758666992187, 'epoch': 2.0})

In [15]:
lora_eval=trainer_lora.evaluate()

In [16]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [17]:
from peft import AutoPeftModelForSequenceClassification
lora_model = AutoPeftModelForSequenceClassification.from_pretrained("gpt-lora")

/home/student/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
base_eval

{'eval_loss': 1.536918044090271,
 'eval_accuracy': 0.486,
 'eval_runtime': 42.0028,
 'eval_samples_per_second': 11.904,
 'eval_steps_per_second': 2.976}

In [19]:
lora_eval

{'eval_loss': 0.694958508014679,
 'eval_accuracy': 0.496,
 'eval_runtime': 47.4966,
 'eval_samples_per_second': 10.527,
 'eval_steps_per_second': 2.632,
 'epoch': 2.0}

Thr lora model has better accuracy than the base model.